# 随机抽样

In [1]:
import tensorflow as tf
import numpy as np
import random

In [15]:
GEN_RANK_NUM = 4+1
LENGTH = 7
batch_size = 3
K = 1

In [3]:
y = tf.convert_to_tensor([False,False,False])
z = tf.convert_to_tensor([True,True,False])

In [4]:
tp = tf.equal(tf.constant(1),tf.reshape(tf.Variable(1),[1,1]))

In [5]:
tpp = tf.convert_to_tensor([True,False,False])

In [6]:
tmp = tf.cond(tf.cast(tf.reduce_sum(tf.cast(y,tf.int32)),tf.bool),lambda:y,lambda:tpp)

In [7]:
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [91]:
sess.run(tpp)

array([ True, False, False])

In [92]:
sess.run(tmp)

array([ True, False, False])

In [8]:
np.random.seed(1)
a = np.random.random_sample((batch_size, LENGTH,GEN_RANK_NUM))
#a = np.reshape(a,[3,70*5])
tensor_a = tf.convert_to_tensor(a)

In [9]:
def sample_set(table, n):  # n=K,从70里面抽取20个sku
    result = []  # 每个样本挑选出k个list
    print('table-shape',table.shape)  # [4,70]
    print('k={}'.format(n))
    cnt = 0
    while len(result)!=n:  # 抽样n个list
        cnt += 1
        print('count:',cnt)
        check_tb.append(table)
        r = sample_one(table.copy())  # [1,4]
        print('sample_set,res',r)
        if  r not in result:
            print("record:",result)
            print('add:',r)
            result.append(r)  # [20, 4]
        
        #if cnt > 3:
        #    break
    return np.array(result)

In [10]:
a

array([[[4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01,
         1.46755891e-01],
        [9.23385948e-02, 1.86260211e-01, 3.45560727e-01, 3.96767474e-01,
         5.38816734e-01],
        [4.19194514e-01, 6.85219500e-01, 2.04452250e-01, 8.78117436e-01,
         2.73875932e-02],
        [6.70467510e-01, 4.17304802e-01, 5.58689828e-01, 1.40386939e-01,
         1.98101489e-01],
        [8.00744569e-01, 9.68261576e-01, 3.13424178e-01, 6.92322616e-01,
         8.76389152e-01],
        [8.94606664e-01, 8.50442114e-02, 3.90547832e-02, 1.69830420e-01,
         8.78142503e-01],
        [9.83468338e-02, 4.21107625e-01, 9.57889530e-01, 5.33165285e-01,
         6.91877114e-01]],

       [[3.15515631e-01, 6.86500928e-01, 8.34625672e-01, 1.82882773e-02,
         7.50144315e-01],
        [9.88861089e-01, 7.48165654e-01, 2.80443992e-01, 7.89279328e-01,
         1.03226007e-01],
        [4.47893526e-01, 9.08595503e-01, 2.93614148e-01, 2.87775339e-01,
         1.30028572e-01],
        

In [11]:
def sample_and_test(prob_table, temperature, k=K):
    if len(prob_table.shape) == 3:
        prob_table = prob_table[:,:,0:-1]
    elif len(prob_table.shape) == 2:
        prob_table = np.reshape(prob_table, (-1, LENGTH, GEN_RANK_NUM))[:,:,0:-1]   
    count = 0
    set_count = 0
    right = 0
    comb = 0
    res = []
    for i in range(prob_table.shape[0]):  # 循环batch_size次，每个样本下[70,4]抽样20个sku在4个位置上的概率
        print('exam:',i)
        table = prob_table[i]  # [70,4]
        table = np.transpose(table)  # [4,70]
        table = np.exp(table * temperature)
        
        
        result = sample_set(table, k)  # [20,5-1]
        res.append(np.array(result))
    print(res)
    return np.array(res)  # [bs, K, GEN_RANK_NUM-1] -> [3,20,4]

In [18]:
def sample_set(table, n):  # n=K,从70里面抽取20个sku
    result = []  # 每个样本挑选出k个list
    print('table-shape',table.shape)  # [4,70]
    print('k={}'.format(n))
    cnt = 0
    num=n
    while num>0:  # 抽样n个list
        cnt += 1
        print('这么多次输入table.copy count:',cnt)
        #check_tb.append(table)
        r = sample_one(table.copy())  # [1,4]
        print('sample_set,res',r)
        if len(result)==0:
            result.append(r)
            num -= 1
            print('第一次加入sample-set list')
        elif r not in result:
            print("record:",result)
            print('add:',r)
            result.append(r)  # [20, 4]
            num-=1
            print('不通的list入栈')
        
        if cnt > 30:
            break
    return np.array(result)

In [22]:
check = []
check_tb = []
def sample_one(table):
    
    np.random.seed(16)
    random_shifts = np.random.random([LENGTH,GEN_RANK_NUM-1]).transpose()  # [4,70]
    #print('------------\n',random_shifts,'-----------\n')
    random_shifts /= random_shifts.sum(axis = 1)[:,np.newaxis]
    result = []
    print('sample_one,table_shape',table.shape)
    for i in range(table.shape[0]):  # 每个位置[0,4),一共4个位置，每个位置有70个候选sku
        print('sample one sum & origin table[i]:',table[i].sum(),table[i])
        table[i] /= table[i].sum() # [1,70]
        print('shifts:',random_shifts[i])
        print('af table[i]:',table[i])
        shifted_probabilities = random_shifts[i] - table[i]
        print('shifted_probabilities:',shifted_probabilities)
        check.append(shifted_probabilities)
        #print('shifted_probabilities.shape:',shifted_probabilities)
        l = np.argpartition(shifted_probabilities, 1, axis=0)[0].astype(np.int64)  # 选出每行最小值的index
        #print('argpartition:',shifted_probabilities[l])
        table[i:,l] = 0
        #print('table[i:,l],i={},l={}:'.format(i,l),table[i:,l])
        result.append(l)
        #print('sample_one',table,l)
    print('finish sample one:',table)
    check_tb.append(table)
    return result

In [23]:
k_list = sample_and_test(a, 0.4)

exam: 0
table-shape (4, 7)
k=2
这么多次输入table.copy count: 1
sample_one,table_shape (4, 7)
sample one sum & origin table[i]: 8.557202840661846 [1.18152834 1.03762603 1.18255554 1.30759164 1.37753797 1.43024058
 1.04012275]
shifts: [0.08899459 0.14377159 0.02802858 0.28801461 0.27764021 0.04817727
 0.12537316]
af table[i]: [0.13807413 0.12125762 0.13819417 0.15280597 0.16097994 0.1671388
 0.12154939]
shifted_probabilities: [-0.04907954  0.02251397 -0.11016559  0.13520864  0.11666027 -0.11896153
  0.00382378]
sample one sum & origin table[i]: 7.5647384313428425 [1.33393043 1.07734974 1.31533028 1.181662   1.47300514 0.
 1.18346083]
shifts: [0.15994603 0.06820224 0.287694   0.04844337 0.14193884 0.28056323
 0.01321229]
af table[i]: [0.1763353  0.14241732 0.17387651 0.15620659 0.1947199  0.
 0.15644438]
shifted_probabilities: [-0.01638927 -0.07421508  0.11381749 -0.10776322 -0.05278106  0.28056323
 -0.14323209]
sample one sum & origin table[i]: 5.617479789954894 [1.00004575 1.14823306 1.085218

  0.        ]]
sample_set,res [5, 6, 3, 2]
这么多次输入table.copy count: 12
sample_one,table_shape (4, 7)
sample one sum & origin table[i]: 8.557202840661846 [1.18152834 1.03762603 1.18255554 1.30759164 1.37753797 1.43024058
 1.04012275]
shifts: [0.08899459 0.14377159 0.02802858 0.28801461 0.27764021 0.04817727
 0.12537316]
af table[i]: [0.13807413 0.12125762 0.13819417 0.15280597 0.16097994 0.1671388
 0.12154939]
shifted_probabilities: [-0.04907954  0.02251397 -0.11016559  0.13520864  0.11666027 -0.11896153
  0.00382378]
sample one sum & origin table[i]: 7.5647384313428425 [1.33393043 1.07734974 1.31533028 1.181662   1.47300514 0.
 1.18346083]
shifts: [0.15994603 0.06820224 0.287694   0.04844337 0.14193884 0.28056323
 0.01321229]
af table[i]: [0.1763353  0.14241732 0.17387651 0.15620659 0.1947199  0.
 0.15644438]
shifted_probabilities: [-0.01638927 -0.07421508  0.11381749 -0.10776322 -0.05278106  0.28056323
 -0.14323209]
sample one sum & origin table[i]: 5.617479789954894 [1.00004575 1.1482

 0.15644438]
shifted_probabilities: [-0.01638927 -0.07421508  0.11381749 -0.10776322 -0.05278106  0.28056323
 -0.14323209]
sample one sum & origin table[i]: 5.617479789954894 [1.00004575 1.14823306 1.08521801 1.25041554 1.13356742 0.
 0.        ]
shifts: [0.15351232 0.19198778 0.15713058 0.06976786 0.05995025 0.15790133
 0.20974988]
af table[i]: [0.17802392 0.20440359 0.19318592 0.22259369 0.20179288 0.
 0.        ]
shifted_probabilities: [-0.0245116  -0.01241581 -0.03605535 -0.15282582 -0.14184263  0.15790133
  0.20974988]
sample one sum & origin table[i]: 5.040454787760884 [1.12854933 1.17199449 1.42083819 0.         1.31907277 0.
 0.        ]
shifts: [0.02232186 0.08014549 0.03817675 0.14366014 0.2187186  0.23105044
 0.26592672]
af table[i]: [0.22389831 0.23251761 0.2818869  0.         0.26169717 0.
 0.        ]
shifted_probabilities: [-0.20157645 -0.15237212 -0.24371015  0.14366014 -0.04297857  0.23105044
  0.26592672]
finish sample one: [[0.13807413 0.12125762 0.13819417 0.1528059

sample_set,res [2, 6, 4, 1]
这么多次输入table.copy count: 4
sample_one,table_shape (4, 7)
sample one sum & origin table[i]: 8.126119013531003 [1.13451614 1.48519256 1.19620903 1.00777687 1.02157445 1.04178311
 1.23906686]
shifts: [0.08899459 0.14377159 0.02802858 0.28801461 0.27764021 0.04817727
 0.12537316]
af table[i]: [0.13961353 0.18276776 0.14720545 0.124017   0.12571493 0.1282018
 0.15247954]
shifted_probabilities: [-0.05061894 -0.03899617 -0.11917687  0.16399761  0.15192528 -0.08002453
 -0.02710637]
sample one sum & origin table[i]: 7.719238536202536 [1.31600466 1.34886873 0.         1.31197576 1.25815586 1.1801274
 1.30410614]
shifts: [0.15994603 0.06820224 0.287694   0.04844337 0.14193884 0.28056323
 0.01321229]
af table[i]: [0.17048374 0.17474116 0.         0.16996181 0.16298963 0.15288132
 0.16894233]
shifted_probabilities: [-0.01053771 -0.10653892  0.287694   -0.12151844 -0.02105079  0.12768191
 -0.15573004]
sample one sum & origin table[i]: 5.984000384233422 [1.39633405 1.118711

  0.        ]]
sample_set,res [2, 6, 4, 1]
这么多次输入table.copy count: 15
sample_one,table_shape (4, 7)
sample one sum & origin table[i]: 8.126119013531003 [1.13451614 1.48519256 1.19620903 1.00777687 1.02157445 1.04178311
 1.23906686]
shifts: [0.08899459 0.14377159 0.02802858 0.28801461 0.27764021 0.04817727
 0.12537316]
af table[i]: [0.13961353 0.18276776 0.14720545 0.124017   0.12571493 0.1282018
 0.15247954]
shifted_probabilities: [-0.05061894 -0.03899617 -0.11917687  0.16399761  0.15192528 -0.08002453
 -0.02710637]
sample one sum & origin table[i]: 7.719238536202536 [1.31600466 1.34886873 0.         1.31197576 1.25815586 1.1801274
 1.30410614]
shifts: [0.15994603 0.06820224 0.287694   0.04844337 0.14193884 0.28056323
 0.01321229]
af table[i]: [0.17048374 0.17474116 0.         0.16996181 0.16298963 0.15288132
 0.16894233]
shifted_probabilities: [-0.01053771 -0.10653892  0.287694   -0.12151844 -0.02105079  0.12768191
 -0.15573004]
sample one sum & origin table[i]: 5.984000384233422 [1.3

...

sample one sum & origin table[i]: 7.824437614159595 [1.05652992 1.44918805 0.         1.18677136 1.46198588 1.43526831
 1.23469409]
shifts: [0.15994603 0.06820224 0.287694   0.04844337 0.14193884 0.28056323
 0.01321229]
af table[i]: [0.1350295  0.18521306 0.         0.15167497 0.18684869 0.18343405
 0.15779972]
shifted_probabilities: [ 0.02491653 -0.11701082  0.287694   -0.1032316  -0.04490984  0.09712917
 -0.14458744]
sample one sum & origin table[i]: 6.132639127609705 [1.05729159 1.14924631 0.         1.47099055 1.19717529 1.25793539
 0.        ]
shifts: [0.15351232 0.19198778 0.15713058 0.06976786 0.05995025 0.15790133
 0.20974988]
af table[i]: [0.17240401 0.18739833 0.         0.23986257 0.19521372 0.20512138
 0.        ]
shifted_probabilities: [-0.01889169  0.00458946  0.15713058 -0.1700947  -0.13526347 -0.04722005
  0.20974988]
sample one sum & origin table[i]: 4.992959082141245 [1.38120529 1.35029737 0.         0.         1.26030763 1.00114879
 0.        ]
shifts: [0.02232186 0.

  0.26592672]
finish sample one: [[0.16603559 0.12359166 0.         0.16553719 0.12111491 0.1271862
  0.13182817]
 [0.1350295  0.18521306 0.         0.15167497 0.18684869 0.18343405
  0.        ]
 [0.17240401 0.18739833 0.         0.         0.19521372 0.20512138
  0.        ]
 [0.         0.2704403  0.         0.         0.25241698 0.20051212
  0.        ]]
sample_set,res [2, 6, 3, 0]
这么多次输入table.copy count: 10
sample_one,table_shape (4, 7)
sample one sum & origin table[i]: 8.644418952017286 [1.43528117 1.06837807 1.42379015 1.43097279 1.04696802 1.09945081
 1.13957793]
shifts: [0.08899459 0.14377159 0.02802858 0.28801461 0.27764021 0.04817727
 0.12537316]
af table[i]: [0.16603559 0.12359166 0.16470629 0.16553719 0.12111491 0.1271862
 0.13182817]
shifted_probabilities: [-0.077041    0.02017993 -0.13667771  0.12247742  0.1565253  -0.07900893
 -0.00645501]
sample one sum & origin table[i]: 7.824437614159595 [1.05652992 1.44918805 0.         1.18677136 1.46198588 1.43526831
 1.23469409]


af table[i]: [0.2766306  0.2704403  0.         0.         0.25241698 0.20051212
 0.        ]
shifted_probabilities: [-0.25430874 -0.19029482  0.03817675  0.14366014 -0.03369838  0.03053832
  0.26592672]
finish sample one: [[0.16603559 0.12359166 0.         0.16553719 0.12111491 0.1271862
  0.13182817]
 [0.1350295  0.18521306 0.         0.15167497 0.18684869 0.18343405
  0.        ]
 [0.17240401 0.18739833 0.         0.         0.19521372 0.20512138
  0.        ]
 [0.         0.2704403  0.         0.         0.25241698 0.20051212
  0.        ]]
sample_set,res [2, 6, 3, 0]
这么多次输入table.copy count: 18
sample_one,table_shape (4, 7)
sample one sum & origin table[i]: 8.644418952017286 [1.43528117 1.06837807 1.42379015 1.43097279 1.04696802 1.09945081
 1.13957793]
shifts: [0.08899459 0.14377159 0.02802858 0.28801461 0.27764021 0.04817727
 0.12537316]
af table[i]: [0.16603559 0.12359166 0.16470629 0.16553719 0.12111491 0.1271862
 0.13182817]
shifted_probabilities: [-0.077041    0.02017993 -0.13

sample_one,table_shape (4, 7)
sample one sum & origin table[i]: 8.644418952017286 [1.43528117 1.06837807 1.42379015 1.43097279 1.04696802 1.09945081
 1.13957793]
shifts: [0.08899459 0.14377159 0.02802858 0.28801461 0.27764021 0.04817727
 0.12537316]
af table[i]: [0.16603559 0.12359166 0.16470629 0.16553719 0.12111491 0.1271862
 0.13182817]
shifted_probabilities: [-0.077041    0.02017993 -0.13667771  0.12247742  0.1565253  -0.07900893
 -0.00645501]
sample one sum & origin table[i]: 7.824437614159595 [1.05652992 1.44918805 0.         1.18677136 1.46198588 1.43526831
 1.23469409]
shifts: [0.15994603 0.06820224 0.287694   0.04844337 0.14193884 0.28056323
 0.01321229]
af table[i]: [0.1350295  0.18521306 0.         0.15167497 0.18684869 0.18343405
 0.15779972]
shifted_probabilities: [ 0.02491653 -0.11701082  0.287694   -0.1032316  -0.04490984  0.09712917
 -0.14458744]
sample one sum & origin table[i]: 6.132639127609705 [1.05729159 1.14924631 0.         1.47099055 1.19717529 1.25793539
 0.   

 0.        ]
shifts: [0.15351232 0.19198778 0.15713058 0.06976786 0.05995025 0.15790133
 0.20974988]
af table[i]: [0.17240401 0.18739833 0.         0.23986257 0.19521372 0.20512138
 0.        ]
shifted_probabilities: [-0.01889169  0.00458946  0.15713058 -0.1700947  -0.13526347 -0.04722005
  0.20974988]
sample one sum & origin table[i]: 4.992959082141245 [1.38120529 1.35029737 0.         0.         1.26030763 1.00114879
 0.        ]
shifts: [0.02232186 0.08014549 0.03817675 0.14366014 0.2187186  0.23105044
 0.26592672]
af table[i]: [0.2766306  0.2704403  0.         0.         0.25241698 0.20051212
 0.        ]
shifted_probabilities: [-0.25430874 -0.19029482  0.03817675  0.14366014 -0.03369838  0.03053832
  0.26592672]
finish sample one: [[0.16603559 0.12359166 0.         0.16553719 0.12111491 0.1271862
  0.13182817]
 [0.1350295  0.18521306 0.         0.15167497 0.18684869 0.18343405
  0.        ]
 [0.17240401 0.18739833 0.         0.         0.19521372 0.20512138
  0.        ]
 [0.     

In [24]:
k_list


array([[[5, 6, 3, 2]],

       [[2, 6, 4, 1]],

       [[2, 6, 3, 0]]], dtype=int64)

In [101]:
k_list.shape

(3, 1, 4)

In [102]:
len(check)

372

In [103]:
check

[array([ 0.12167263, -0.08993863, -0.04061886, -0.0021012 ,  0.07752943,
        -0.15129132,  0.08474796]),
 array([-0.08899943,  0.01412652, -0.01010667, -0.00392287, -0.08601671,
         0.18905611, -0.01413695]),
 array([ 0.19378642, -0.06369776, -0.07497371, -0.05322852,  0.02046492,
         0.18928915, -0.2116405 ]),
 array([ 0.00886328, -0.14680542, -0.12947867, -0.00235562, -0.09032164,
         0.15653493,  0.20356314]),
 array([ 0.12167263, -0.08993863, -0.04061886, -0.0021012 ,  0.07752943,
        -0.15129132,  0.08474796]),
 array([-0.08899943,  0.01412652, -0.01010667, -0.00392287, -0.08601671,
         0.18905611, -0.01413695]),
 array([ 0.19378642, -0.06369776, -0.07497371, -0.05322852,  0.02046492,
         0.18928915, -0.2116405 ]),
 array([ 0.00886328, -0.14680542, -0.12947867, -0.00235562, -0.09032164,
         0.15653493,  0.20356314]),
 array([ 0.12167263, -0.08993863, -0.04061886, -0.0021012 ,  0.07752943,
        -0.15129132,  0.08474796]),
 array([-0.08899943

In [119]:
check_tb

[array([[1.18152834, 1.03762603, 1.18255554, 1.30759164, 1.37753797,
         1.43024058, 1.04012275],
        [1.33393043, 1.07734974, 1.31533028, 1.181662  , 1.47300514,
         1.0346029 , 1.18346083],
        [1.00004575, 1.14823306, 1.08521801, 1.25041554, 1.13356742,
         1.01574457, 1.46690657],
        [1.12854933, 1.17199449, 1.42083819, 1.05776139, 1.31907277,
         1.07029271, 1.23771398]]),
 array([[0.13807413, 0.12125762, 0.13819417, 0.15280597, 0.        ,
         0.1671388 , 0.12154939],
        [0.1871832 , 0.15117863, 0.        , 0.1658162 , 0.        ,
         0.1451802 , 0.16606862],
        [0.1700369 , 0.19523306, 0.        , 0.21260705, 0.        ,
         0.17270616, 0.        ],
        [0.2548322 , 0.26464233, 0.        , 0.        , 0.        ,
         0.24167755, 0.        ]]),
 array([[1.18152834, 1.03762603, 1.18255554, 1.30759164, 1.37753797,
         1.43024058, 1.04012275],
        [1.33393043, 1.07734974, 1.31533028, 1.181662  , 1.47300514,


In [42]:
def set_value(matrix, x, y, col_len=4,val=0.):
    # 提取出要更新的行
    row = tf.gather(matrix, x)
    # 构造这行的新数据
    new_row = tf.concat([row[:y], [val]*(col_len-y)], axis=0)
    # 使用 tf.scatter_update 方法进正行替换
    #matrix.assign(tf.scatter_update(matrix, x, new_row))  
    return tf.scatter_update(matrix, x, new_row)

In [131]:
def tf_sample_one(table):
    np.random.seed(0)
    random_shifts = np.random.random([LENGTH,GEN_RANK_NUM-1])  # [70,4]
    random_shifts /= random_shifts.sum(axis = 1)[:,np.newaxis]
    result = []
    for m in range(GEN_RANK_NUM-1):  # 4个位置
        table_m = table[:,m]  # [70,1]
        table_m /= tf.reduce_sum(table_m)
        shifted_probabilities = random_shifts[:,m] - table_m
        l = tf.argmin(shifted_probabilities)  # 找出最小的index
        table = set_value(table,x=l,y=m,col_len=GEN_RANK_NUM-1,val=0.)
        result.append(l)
    return result
        

In [46]:
def tf_check_repeat(checked, cand_set):
    flag = tf.convert_to_tensor(False)
    for cand_tensor in cand_set:
        flag = tf.cond(checked==cand_tensor, lambda: True, lambda: flag)
    return flag  # 但凡有一个重复则true

In [129]:
def tf_sample_and_test(prob_table, temperature, k=K):
    prob_table = tf.convert_to_tensor(prob_table)
    if len(prob_table.shape) == 3:
        print('old_shape:',prob_table.shape)
        prob_table = prob_table[0:,0:,:-1]
        print('new_shape:',prob_table.shape)
    elif len(prob_table.shape) == 2:
        prob_table = np.reshape(prob_table, (-1, LENGTH, GEN_RANK_NUM))[:,:,0:-1]   
    print('prob_table.shape:',prob_table.shape)
    res = []
    prob_table = tf.math.exp(prob_table * temperature)  # [bs, LENGTH, GEN_RANK_NUM-1]  [3,70,4]
    
    # 从每条候选概率[70,4]中抽样20个即[20,4]
    for i in range(prob_table.get_shape().as_list()[0]):  # bs
        sample_set = []
        
        while k > 0:  # 抽取20个sku的位置概率
            table  = tf.Variable(prob_table[i])  # 每次都从原始的数据开始
            print('table:',table)
            tmp = tf_sample_one(table)  # [70,4] -> [1,4]
            if len(sample_set)==0:
                sample_set_concat = tmp
                k -= 1
            else:
                tmp_len = sample_set_concat.get_shape().as_list()[0]
                tf.cond(check_repeat(tmp,sample_set), 
                        true_fn=sample_set_concat, 
                        false_fn=tf.concat([sample_set_concat,emp],0))  
                if sample_set_concat.get_shape().as_list()[0]!=tmp:
                    sample_set.append(tmp)
                    k -= 1
        res.append(sample_set_concat)
    res = tf.concat(res,0)
    return res  # [bs, K, GEN_RANK_NUM-1]

In [188]:
emb = tf.nn.embedding_lookup(output,k_list)
emb

<tf.Tensor 'embedding_lookup_1/Identity:0' shape=(3, 20, 4, 10, 4) dtype=float64>

In [187]:
emb_lookup = sess.run(emb)

InvalidArgumentError: indices[0,0,0] = 9 is not in [0, 3)
	 [[node embedding_lookup (defined at d:\python\anaconda3\envs\tf15\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]

Original stack trace for 'embedding_lookup':
  File "d:\python\anaconda3\envs\tf15\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\python\anaconda3\envs\tf15\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\python\anaconda3\envs\tf15\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\python\anaconda3\envs\tf15\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "d:\python\anaconda3\envs\tf15\lib\site-packages\ipykernel\kernelapp.py", line 597, in start
    self.io_loop.start()
  File "d:\python\anaconda3\envs\tf15\lib\site-packages\tornado\platform\asyncio.py", line 149, in start
    self.asyncio_loop.run_forever()
  File "d:\pyt

In [176]:
def tensor_sample(prob_table_tensor,temperature,k):
    res=tf.py_func(sample_and_test,[prob_table_tensor,temperature,k],tf.int64)
    return res

In [177]:
sess = tf.Session()

In [178]:
ts=tensor_sample(tf.convert_to_tensor(output),tf.convert_to_tensor(0.4),tf.convert_to_tensor(20))

In [179]:
ts=tensor_sample(output,0.4,3)

In [180]:
qweq= sess.run(ts)
qweq

table-shape (4, 10)
sample_one,table_shape (4, 10)
sample_one [[0.08126751 0.09057037 0.09669056 0.10559904 0.08597578 0.09663535
  0.08883014 0.11984099 0.11865811 0.        ]
 [1.36885185 1.06306624 1.36478918 1.10353117 1.44675058 1.13381408
  1.00779503 1.1847161  1.08729835 0.        ]
 [1.19280543 1.03363841 1.24784704 1.10620105 1.12694515 1.02030012
  1.20366878 1.00831026 1.10047868 0.        ]
 [1.13351313 1.07931692 1.33611616 1.41682762 1.09413343 1.39556951
  1.16789658 1.35496673 1.05787337 0.        ]] 9
sample_one [[0.08126751 0.09057037 0.09669056 0.10559904 0.08597578 0.09663535
  0.08883014 0.11984099 0.11865811 0.        ]
 [0.12720947 0.09879235 0.12683192 0.10255282 0.13444872 0.10536706
  0.09365592 0.11009746 0.         0.        ]
 [1.19280543 1.03363841 1.24784704 1.10620105 1.12694515 1.02030012
  1.20366878 1.00831026 0.         0.        ]
 [1.13351313 1.07931692 1.33611616 1.41682762 1.09413343 1.39556951
  1.16789658 1.35496673 0.         0.        ]] 8
s

array([[[9, 8, 2, 5],
        [8, 5, 2, 1],
        [9, 4, 0, 2]],

       [[6, 5, 8, 3],
        [2, 6, 3, 0],
        [7, 6, 8, 3]],

       [[8, 1, 4, 0],
        [3, 7, 2, 5],
        [0, 1, 3, 6]]], dtype=int64)

In [182]:
qweq.shape

(3, 3, 4)